In [ ]:
import pandas as pd


queries = pd.read_csv('queries/queries_with_spark_llama_gpt.csv')


In [ ]:
# Calcular a quantidade de False por difficulty
counts = queries['difficulty'].value_counts()
# Criar um DataFrame com os resultados
counts = counts.reset_index()
counts.columns = ['Nível de Dificuldade', 'Quantidade de queries']

In [ ]:
counts

In [ ]:
import re

# Função para eliminar os textos especificados de uma frase
def eliminar_textos_longos(frase):
    # Textos a serem eliminados
    textos_a_eliminar = [
        "TOTAL_SOCIOS ELIMINAR",
        "TOTAL_SOCIOS_COM_PAIS",
        "TOTAL_SOCIOS_DIRETOR",
        "TOTAL_SOCIOS_PRESIDENTE",
        "TOTAL_SOCIOS_QUALIFICADOS",
        "TOTAL_SOCIOS",
        "TOTAL_EMPRESAS",
        "QTD_SOCIOS",  
        "TOTAL_ESTABELECIMENTOS",
        "_ADMINISTRADORES",
        "_COM_NOME",
        "_FAIXA_ETARIA",        
        "SELECT",
        '\"PUBLIC\"',
        'PUBLIC',
        "UPPER",
        "TOTAL"
    ]
    
    # Eliminar os textos da frase
    for texto in textos_a_eliminar:
        frase = frase.replace(texto, "")

    
    return frase

def eliminar_textos_curtos(frase):
    # Textos a serem eliminados
    textos_a_eliminar = [
        " AS ", ";", " EM ", '"',"'"," EST "," S "," SIM "," EMP "," E ",")","("
    ]
    
    # Eliminar os textos da frase
    for texto in textos_a_eliminar:
        frase = frase.replace(texto, " ")
        
    return frase

def substituir_espaco(frase):
    # Textos a serem eliminados
    textos_a_eliminar = [
        "(EM.", "(EST.","(S.","(SIM.","(EMP.","(E.","EM.", "EST.","S.","SIM.","EMP.","E."," ."
    ]
    
    # Eliminar os textos da frase
    for texto in textos_a_eliminar:
        frase = frase.replace(texto, " ")
        
    return frase


In [ ]:
import re

def preprocessa_query(querie):
    # Converte a consulta para letras maiúsculas
    querie = querie.upper()
    querie= querie.replace("'SIM'", "'POSITIVO'")
    querie= querie.replace("DATE_PART('YEAR',", "EXTRACT(YEAR FROM")
    querie= querie.replace("<>", "!=")
    querie = eliminar_textos_longos(querie)
    querie = eliminar_textos_curtos(querie)
    querie = substituir_espaco(querie)

    # Divide a consulta em palavras
    querie = querie.split()
    return querie


In [ ]:
# Função para percorrer o DataFrame e comparar as consultas
def process_and_compare_queries(df):
    gold_querie = []
    generated_querie = []
    comparison_results = []

    for index, row in df.iterrows():
        gold = preprocessa_query(row['postgres_query'])
        generated = preprocessa_query(row['query_llamma_GPT'])
        
        gold_querie.append(gold)
        generated_querie.append(generated)
        
        # Comparar os vetores independentemente da ordem
        if sorted(gold) == sorted(generated):
            comparison_results.append(True)
        else:
            comparison_results.append(False)
    
    df['gold_querie'] = gold_querie
    df['generated_querie'] = generated_querie
    df['comparison_results'] = comparison_results

    return df

queries = process_and_compare_queries(queries)

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

queries_true = queries[queries['comparison_results'] == True]

# Display the filtered DataFram
queries_true

In [ ]:
queries.to_csv('queries/queries_final.csv')

In [ ]:
# Filtrar as colunas 'difficulty' e 'comparison_results'
filtered_df = queries[['difficulty', 'comparison_results']]
filtered_df = filtered_df[queries['difficulty'] != 'Extra Hard']

# Calcular a quantidade de False por difficulty
false_counts = filtered_df[filtered_df['comparison_results'] == False]['difficulty'].value_counts()
true_counts = filtered_df[filtered_df['comparison_results'] == True]['difficulty'].value_counts()
percent_true = (true_counts*100)/(true_counts+false_counts)

# Criar um DataFrame com os resultados
false_counts_df = false_counts.reset_index()
false_counts_df.columns = ['difficulty', 'false_counts']

# Exibir a tabela
false_counts_df

In [ ]:
true_counts_df = true_counts.reset_index()
true_counts_df.columns = ['difficulty', 'true_counts']
true_counts

In [ ]:
percent_true_counts_df = percent_true.reset_index()
percent_true_counts_df.columns = ['difficulty', 'true_counts']
percent_true

In [ ]:
# Filtrar as colunas 'difficulty' e 'comparison_results'
filtered_df = queries[['explanation', 'comparison_results']]
filtered_df = filtered_df[queries['explanation'] != 'Extra Hard']
# Calcular a quantidade de False por difficulty
false_counts = filtered_df[filtered_df['comparison_results'] == False]['explanation'].value_counts()
true_counts = filtered_df[filtered_df['comparison_results'] == True]['explanation'].value_counts()
true_counts_df = true_counts.reset_index()
true_counts_df.columns = ['explanation', 'true_counts']
true_counts

In [ ]:
# Criar um DataFrame com os resultados
false_counts_df = false_counts.reset_index()
false_counts_df.columns = ['difficulty', 'false_counts']

# Exibir a tabela
false_counts_df